In [1]:
import torch

In [2]:
import torchvision

In [3]:
train_dataset = torchvision.datasets.CIFAR10(root="cifar_train", train=True,
                                      transform=torchvision.transforms.Compose(
                                          [
                                              torchvision.transforms.RandomHorizontalFlip(),
                                              torchvision.transforms.ToTensor()
                                          ]
                                      ),
                                       download=True
                                      )

Files already downloaded and verified


In [4]:
val_dataset = torchvision.datasets.CIFAR10(root="cifar_val", train=False,
                                      transform=torchvision.transforms.Compose(
                                          [
                                              torchvision.transforms.RandomHorizontalFlip(),
                                              torchvision.transforms.ToTensor()
                                          ]
                                      ),
                                       download=True
                                      )

Files already downloaded and verified


In [5]:
dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=False)

In [6]:
for batch in dataloader:
    pass

In [9]:
batch[0].shape, batch[1].shape

(torch.Size([80, 3, 32, 32]), torch.Size([80]))

In [5]:
import sys
sys.path.append("/Users/leo/projects/generalization_lipschitz")

In [6]:
from robustness.split_partitions import *

In [7]:
centroids = select_partition_centroid(500, train_dataset)

In [8]:
centroids.shape

torch.Size([500, 3072])

In [9]:
indices = assign_partition(val_dataset, centroids)

In [10]:
indices

tensor([105, 281,  30,  ..., 260, 105, 419])

In [11]:
model_checkpoint = torch.load("model_best.pth.tar", map_location="cpu")

In [12]:
from model._mlp import _mlp

In [13]:
model = _mlp(in_features=3*32*32, cfg=[1024, 512, 256, 64], norm_layer=None, num_classes=10)

In [14]:
model.load_state_dict(model_checkpoint['state_dict'])

<All keys matched successfully>

In [15]:
calculate_robustness(model=model, test_dataset=val_dataset, indices=indices, loss_func=torch.nn.CrossEntropyLoss(reduction='none'))

11.98582935333252